In [1]:
import urllib

import mdtraj as md
import nglview
import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit
from openff.units.openmm import to_openmm
from openmm import app
from openmm import unit as openmm_unit

from openff.interchange.components.interchange import Interchange
from openff.interchange.drivers import get_amber_energies, get_openmm_energies
from openff.interchange.drivers.all import get_summary_data

In [2]:
class _Molecule(Molecule):
    @classmethod
    def from_pdb(self, file_path):
        molecule = Molecule.from_pdb(file_path)
        molecule._add_default_hierarchy_schemes()
        molecule.perceive_hierarchy()
        return molecule


class _Topology(Topology):
    from openmm import app

    @property
    def positions(self):
        return np.vstack([mol.conformers[0] for mol in self.molecules])

    def to_file(self, filename, positions):
        with open(filename, "w") as f:
            app.PDBFile.writeFile(
                self.to_openmm(ensure_unique_atom_names=False),
                positions,
                f,
            )


def get_packed_coordinates(structure: str, n_waters: int):
    from openff.evaluator import unit as evaluator_unit
    from openff.evaluator.utils.packmol import pack_box

    water = Molecule.from_smiles("O")
    water.generate_conformers(n_conformers=1)

    trj = md.load(structure)

    if trj.unitcell_lengths:
        box_size = 1.2 * trj.unitcell_lengths
    else:
        box_size = 1.2 * (np.max(trj.xyz, axis=1) - np.min(trj.xyz, axis=1))[0]

    packed_trj, _ = pack_box(
        molecules=[water],
        number_of_copies=[n_waters],
        structure_to_solvate=structure,
        box_size=box_size * evaluator_unit.nanometer,
        # mass_density=0.8 * evaluator_unit.Unit("g/cm**3"),
    )

    return packed_trj.xyz[0]

This example uses sample data from [Protein Ligand Benchmark](https://github.com/openforcefield/protein-ligand-benchmark#proteinligandbenchmarks) data set curated by the Open Force Field Initiative. Specifially, [MCL1](https://github.com/openforcefield/protein-ligand-benchmark/tree/8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample) data is used. Conveniently for the purposes of this example, the ligand is already docked and the protein is relatively small (~2000 atoms). Follow the links for details or to swap out ligand(s).

In [3]:
url = (
    "https://raw.githubusercontent.com/openforcefield/protein-ligand-benchmark/"
    "8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample/"
)

urllib.request.urlretrieve(url + "/01_protein/crd/protein.pdb", "protein.pdb")
urllib.request.urlretrieve(url + "02_ligands/lig_23/crd/lig_23.sdf", "lig_23.sdf")

# These two files (`protein.pdb` and `lig_23.sdf`) should be in the local path now
!ls -lhrt

total 5368
-rw-r--r--  1 mwt  staff   2.8K Nov  4 09:51 prepare.py
-rw-r--r--  1 mwt  staff    27K Nov  4 12:23 env.txt
-rw-r--r--  1 mwt  staff   320K Nov  4 14:56 out.gro
-rw-r--r--  1 mwt  staff   946K Nov  4 15:38 out.top
-rw-r--r--  1 mwt  staff   192K Dec 13 17:21 sliced.pdb
-rw-r--r--  1 mwt  staff   248K Dec 13 17:23 docked.pdb
-rw-r--r--  1 mwt  staff   252K Dec 13 17:23 packed.pdb
-rw-r--r--  1 mwt  staff   252K Dec 13 17:46 out.pdb
-rw-r--r--  1 mwt  staff    89K Dec 13 17:48 out.inpcrd
-rw-r--r--  1 mwt  staff    90B Dec 13 17:48 out.prmtop
-rw-r--r--  1 mwt  staff   152B Dec 13 17:53 README.md
-rw-r--r--  1 mwt  staff    17K Dec 13 17:55 protein_ligand.ipynb
-rw-r--r--  1 mwt  staff   193K Dec 13 17:55 protein.pdb
-rw-r--r--  1 mwt  staff   4.0K Dec 13 17:55 lig_23.sdf


The PDB file includes a few waters; the OpenFF Toolkit currently does not explicitly support parsing multi-component PDB files, so we'll use [MDTraj](https://mdtraj.org/) to parse the protein and save it to a new file.

In [4]:
protein_with_waters = md.load("protein.pdb")
protein_pdb = protein_with_waters.atom_slice(
    protein_with_waters.top.select("chainid 0")
)
protein_pdb.save("sliced.pdb")

Now, we can use the OpenFF Toolkit to load the protein and ligand from PDB and SDF files, respectively

In [5]:
%%capture
protein = _Molecule.from_pdb("sliced.pdb")
ligand = _Molecule.from_file("lig_23.sdf")

From these `Molecule` objects, we can make a `Topology` object and write it out to a PDB file for visualization

In [6]:
docked_topology = _Topology.from_molecules([protein, ligand])
docked_topology.to_file(
    filename="docked.pdb", positions=to_openmm(docked_topology.positions)
)

In [7]:
from openmm import app

with open("out.pdb", "w") as f:
    app.PDBFile.writeFile(
        docked_topology.to_openmm(ensure_unique_atom_names=False),
        to_openmm(docked_topology.positions),
        f,
    )

In [8]:
w = nglview.show_mdtraj(md.load("docked.pdb"))
w.add_representation(
    "spacefill",
    selection=[*range(protein.n_atoms, docked_topology.n_atoms)],
    color="green",
)
w

NGLWidget()

In [9]:
w.render_image()

Image(value=b'', width='99%')

Next, let's add an arbitrary number of waters to the system and visualize the result. The density here will be wrong; use your imagination to act like the right number of waters were added.

In [10]:
water = Molecule.from_smiles("O")
water.generate_conformers(n_conformers=1)
n_waters = 10

packed_coordinates = get_packed_coordinates("docked.pdb", n_waters)

final_topology = _Topology.from_molecules([protein, ligand, *n_waters * [water]])
final_topology.to_file(
    filename="packed.pdb",
    positions=packed_coordinates * openmm_unit.nanometer,
)

In [11]:
w = nglview.show_mdtraj(md.load("packed.pdb"))
w.add_representation(
    "spacefill",
    selection=[*range(protein.n_atoms, docked_topology.n_atoms)],
    color="green",
)
w

NGLWidget()

In [12]:
w.render_image()

Image(value=b'', width='99%')

Now that we've prepared the topology of the system, we can apply force fields and generate inputs for simulation engines. Here, we'll use [OpenFF 2.0.0 "Sage"](https://openforcefield.org/community/news/general/sage2.0.0-release/) as a small molecule force field for the ligand and [OpenFF's port of Amber's ff14SB](https://github.com/openforcefield/amber-ff-porting/releases/tag/0.0.1) for the protein. Sage happens to include TIP3P parameters which we'll use for the waters. Because of some bugs/performance issues, we have to remote the improper torsions from the protein force field and constraints from both force fields for now.

In [13]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.1.offxml")
ff14sb.deregister_parameter_handler("ImproperTorsions")

sage = ForceField("openff_unconstrained-2.0.0.offxml")

For now, OpenFF's force field lines are not unified; in the future a self-consistent force field can describe both biopolymers and small molecules in one pass. But until then, we need to apply each force field to their respective components, generating an `Interchange` object for each, and then combine them using the `+` operator. This operatator uses custom code that attempts to handle combining the chemical topologies, physical forces, and positions; it's not haphazardly squishing the object together. (In this example, we're setting the positions on each topology before adding them together and then overwriting those positions later using the packed results. This is to get around a bug that  has not been fixed yet.) However, this is still a sharp edge and likely to produce strange behavior - please do not use it in production work!

In [14]:
protein_interchange = Interchange.from_smirnoff(ff14sb, protein.to_topology())

Starting handler: Bonds...
Finished handler: ... Bonds. Took 103.46908330917358
Starting handler: Angles...
Finished handler: ... Angles. Took 31.57309103012085
Starting handler: ProperTorsions...
Finished handler: ... ProperTorsions. Took 71.87460684776306
Starting handler: vdW...
Finished handler: ... vdW. Took 13.755638122558594
Starting handler: Electrostatics...
Finished handler: ... Electrostatics. Took 86.46816682815552


In [15]:
sage_interchange = Interchange.from_smirnoff(
    sage, _Topology.from_molecules([ligand, *n_waters * [water]])
)

Starting handler: Bonds...
Finished handler: ... Bonds. Took 18.595205783843994
Starting handler: Constraints...
Finished handler: ... Constraints. Took 0.03400921821594238
Starting handler: Angles...
Finished handler: ... Angles. Took 0.026002168655395508
Starting handler: ProperTorsions...
Finished handler: ... ProperTorsions. Took 0.12237405776977539
Starting handler: ImproperTorsions...
Finished handler: ... ImproperTorsions. Took 0.03875303268432617
Starting handler: vdW...
Finished handler: ... vdW. Took 0.03524518013000488
Starting handler: Electrostatics...
Finished handler: ... Electrostatics. Took 6.1344451904296875


Since we have already prepared the positions of the final system, which contains all components, we won't track positions in the intermediate `Interchange` objects and instead just use the setter on the final object. This will produce a warning (`Setting positions to None ...`) but that's fine.

In [ ]:
combined_interchange = protein_interchange + sage_interchange
combined_interchange.positions = packed_coordinates * unit.nanometer
combined_interchange.box = [10, 10, 10]  # Arbitrary

/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:576: UserWarning: Iterchange object combination is experimental and likely to produce strange results. Use with caution!
  warnings.warn(


Now that we've prepared all atomic positions, applied each force field, and combined the results, we can visualize the result to verify that at least the positions and topology are not mangled:

In [ ]:
combined_interchange.to_pdb(file_path="out.pdb")

w = nglview.show_mdtraj(md.load("out.pdb"))
w.add_representation(
    "spacefill",
    selection=[*range(protein.n_atoms, docked_topology.n_atoms)],
    color="green",
)
w

In [ ]:
w.render_image()

Finally, we can export the final `Interchange` object to models understood by various simulation engines.

In [ ]:
openmm_system = combined_interchange.to_openmm()
openmm_topology = combined_interchange.topology.to_openmm()
print(type(openmm_system), type(openmm_topology))

In [ ]:
combined_interchange.to_inpcrd("out.inpcrd")
combined_interchange.to_prmtop("out.prmtop")

In [ ]:
combined_interchange.to_gro("out.gro")
combined_interchange.to_top("out.top")

In [ ]:
combined_interchange.to_lammps("out.lmp")

In order to verify the accuracy of each export, we can use functions in the `drivers` module to call out to each engine to evaluate single-point energies. Under the hood, each function uses the export functions just as we did in the above cells. 

In [ ]:
print(get_openmm_energies(combined_interchange))
print(get_amber_energies(combined_interchange))

Note that some of these functions are not yet performant for systems of this size, so we are only evaluating the OpenMM and Amber interfaces. In the future, GROMACS and LAMMPS exports can be included above, and the function `get_summary_data` can be called on it. As a sneak peek, below is the result of calling that function on an `Interchange` that contains only the ligand. The data is presented as a Pandas DataFrame, which incldues convenient methods for summary statistics.

In [ ]:
ligand_interchange = Interchange.from_smirnoff(sage, ligand.to_topology())
ligand_interchange.positions = ligand.conformers[0]
ligand_interchange.box = [10, 10, 10]

In [ ]:
summary = get_summary_data(ligand_interchange)
summary

In [ ]:
summary.describe()